In [2]:
!pip install tweepy transformers torch ipywidgets matplotlib pandas streamlit python-dotenv

  Using cached tweepy-4.16.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached torch-2.8.0-cp313-cp313-win_amd64.whl.metadata (30 kB)
  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached oauthlib-3.3.1-py3-none-any.whl.metadata (7.9 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
Using cached tweepy-4.16.0-py3-none-any.whl (98 kB)
Using cached torch-2.8.0-cp313-cp313-win_amd64.whl (241.3 MB)
Using cached ipywidgets-8.1.7-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl (216 kB)
Using cached oauthlib-3.3.1-py3-none-any.whl (160 kB)
Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl (24 kB)
Using cached networkx-3.5-py3-none-any.whl (2.0 MB)



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import os
import re
import torch
import pandas as pd
import matplotlib.pyplot as plt
import tweepy
from transformers import pipeline
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display, clear_output

In [13]:
load_dotenv()
bearer_token = os.getenv("BEARER_TOKEN")

if not bearer_token:
    raise ValueError("Twitter API Bearer Token not found!")

In [14]:
client = tweepy.Client(bearer_token=bearer_token)

In [15]:
def clean_tweet(tweet):
    tweet = re.sub(r'http\S+', '', tweet)
    tweet = re.sub(r'@[\w]+', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tweet = re.sub(r'[^A-Za-z\s]', '', tweet)
    return tweet.strip()

In [16]:
device = 0 if torch.cuda.is_available() else -1
sentiment_model = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment",
    device=device
)

def map_sentiment(label):
    if "1" in label or "2" in label:
        return "Negative"
    elif "3" in label:
        return "Neutral"
    else:
        return "Positive"

Device set to use cpu


In [17]:
def fetch_and_analyze(query, max_results=20):
    tweets = client.search_recent_tweets(query=query, max_results=max_results, tweet_fields=["lang"])
    results = []
    if tweets.data:
        for tweet in tweets.data:
            if tweet.lang == "en":
                cleaned = clean_tweet(tweet.text)
                if cleaned.strip():
                    prediction = sentiment_model(cleaned)[0]
                    results.append({
                        "Tweet": tweet.text,
                        "Cleaned": cleaned,
                        "BERT_Label": prediction['label'],
                        "Sentiment": map_sentiment(prediction['label']),
                        "Score": round(prediction['score'], 3)
                    })
    return pd.DataFrame(results)

In [19]:
topic_input = widgets.Text(
    value='competitive programming',
    placeholder='Enter a topic/keyword',
    description='Topic:',
    layout=widgets.Layout(width="400px")
)

num_tweets_slider = widgets.IntSlider(
    value=15,
    min=5,
    max=50,
    step=1,
    description='Tweets:',
    continuous_update=False
)

analyze_button = widgets.Button(
    description="Analyze Sentiment",
    button_style='success'
)

output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output()
        print(f"🔎 Fetching tweets for: {topic_input.value} ...")
        df = fetch_and_analyze(topic_input.value, max_results=num_tweets_slider.value)
        
        if not df.empty:
            display(df)

            sentiment_counts = df["Sentiment"].value_counts()
            fig, ax = plt.subplots()
            sentiment_counts.plot(kind="bar", ax=ax, color="skyblue", edgecolor="black")
            ax.set_title("Sentiment Distribution")
            ax.set_xlabel("Sentiment")
            ax.set_ylabel("Count")
            plt.show()
        else:
            print("⚠️ No tweets found for this topic.")

analyze_button.on_click(on_button_clicked)

ui = widgets.VBox([topic_input, num_tweets_slider, analyze_button, output])
display(ui)